In [2]:
import spacy
from spacy.tokens import Doc, Token
nlp = spacy.load("en_core_web_lg")

import numpy as np
import re
import importlib
import time
from typing import Optional

import ogree_wiki as wiki
importlib.reload(wiki)
import items.tools as tools
import scrapping

In [3]:
ACTIONS_DEFAULT = {
                    "ACTION_POSITIVE" : ["make","build","put","place","add","insert"],
                    "ACTION_NEGATIVE" : ["remove", "delete"], 
                    "ALTERATION" : ["modify", "change","move","set","rename","rotate"]
                    }

ACTIONS_CLI = {
                "ACTION_POSITIVE" : "+",
                "ACTION_NEGATIVE" : "-"
                }

SIMILARITY_THRESHOLD = 0.5

PARAMETERS_DICT = {
            "name" : ["name","called"],
            "position" : ["position","at","located","posU","centered","centerXY","startPosition","startPos","endPosition",],
            "rotation" : ["rotation","turned","degree"],
            "size" : ["size","dimensions","height","sizeU","sizeXY"],
            "template" : ["template"],
            "axisOrientation" : ["axisOrientation", "axis", "orientation"],
            "unit" :  ["unit","floorUnit","meter","m","tile","t","foot","f"],
            "slot" : ["slot"],
            "color" : ["color","usableColor","reservedColor","technicalColor"] + list(wiki.COLORS_HEX_BASIC.keys()), 
            "side" : ["side"],
            "temperature" : ["temperature","cold","warm"],
            "type" : ["type","wall","wireframe","plain"],
            # "reserved" : ["reserved"],
            # "technical" : ["technical"]
            }

# TODO : do a default value dict

In [ ]:
def findIndexMainSubject(processed_entry : Doc, dictioIndexKeyWords : dict, indexAction : int, indexMainEntity : int = None) -> int :

    def searchSubjectRecursive(processed_entry : Doc, currentIndex : int, testConformity, level : int = 0) -> (list|None):
        if level == 5 :
            return None
        if testConformity(processed_entry[currentIndex]) :
            return [(currentIndex, level)]
        else:
            childList = []
            for child in processed_entry[currentIndex].children :
                childResult = searchSubjectRecursive(processed_entry, child.i, testConformity, level+1)
                if childResult != None :
                    childList.extend(childResult)
            if bool(childList) == True : # if the list is not empty    
                minValue = min(childList, key=lambda x: x[1])[1]
                return [x for x in childList if x[1] == minValue] 
            return childList
    
    def testConformity(token : Token) -> bool :
        if ((token.i in dictioIndexKeyWords.keys() or token.pos_ == "NOUN") 
            and token.pos_ != "VERB"
            and not token.is_upper) :
            return True
        return False
    
    actionType = dictioIndexKeyWords[indexAction]

    result = searchSubjectRecursive(processed_entry, indexAction, testConformity)
    result = [x[0] for x in result]
    resultLength = len(result)

    if resultLength == 0 :
        raise Exception("Main request not identified")
    elif resultLength == 1 :
        return result[0]
    else : # if there are at least two tokens at the same length from the action

        if actionType == "ALTERATION" : # if alteration, prioritize known parameters
            resultOnlyParameters = [index for index in result if dictioIndexKeyWords[index] in PARAMETERS_DICT.keys()]
            if bool(resultOnlyParameters) :
                return resultOnlyParameters[0]
            else :
                return result[0]
            
        else : # prioritize entities
            resultOnlyEntity = [index for index in result if dictioIndexKeyWords[index] == "entity"]
            if not bool(resultOnlyEntity) :
                return result[0]
            elif len(resultOnlyEntity) == resultLength and indexMainEntity in result :
                return indexMainEntity
            else :
                return resultOnlyEntity[0]


In [ ]:
# test if the relation is coherent, according to the relation
def testRelation(index1 : int, index2 : int, relationType : str, hierarchyPosition : dict) -> bool :
    if relationType == "hierarchy" and hierarchyPosition[index2] < hierarchyPosition[index1]:
        return True
    elif relationType == "location" and hierarchyPosition[index2] == hierarchyPosition[index1]:
        return True
    return False

def findIndexMainEntity(processed_entry : Doc, dictEntities : dict, indexAction : int) -> int :
    counter = 0
    currentIndexes = {index:index for index in dictEntities.keys()}
    currentWords = {index:processed_entry[index] for index in currentIndexes.keys()}

    while (not indexAction in currentIndexes.values()) and counter < 3 :
        currentWords = {originIndex : processed_entry[currentIndex].head for originIndex,currentIndex in currentIndexes.items()}
        currentIndexes = {originIndex : currentWords[originIndex].i for originIndex,_ in currentIndexes.items()}

        if list(currentIndexes.values()).count(indexAction) == 1 :
            return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]
        counter += 1

    if counter == 3 :
        raise Exception("Main entity not found")
    
    if list(currentIndexes.values()).count(indexAction) != 1 :
        listIndexesRemaining = [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction and originIndex > indexAction]
        return listIndexesRemaining[0]
    else :
        return [originIndex for originIndex,currentIndex in currentIndexes.items() if currentIndex == indexAction][0]


def findRelations(processed_entry : Doc, dictEntities : dict, indexAction : int) -> dict :

    RELATIONS = {
        "hierarchy" : ["in", "inside", "of"],
        "location" : ["next"]
    }
    
    dictRelations = {index : None for index in dictEntities.keys()} # empty dict that will be filled

    # go through the ancestors and check if there's a synonym of the relation key words
    for index in dictEntities.keys() :
        for ancestor in processed_entry[index].ancestors :
            for relation in RELATIONS.keys() :
                if max([ancestor.similarity(nlp(word)[0]) > SIMILARITY_THRESHOLD for word in RELATIONS[relation]]) :
                    dictRelations[index] = relation
                    break

    global INDEX_MAIN_ENTITY

    # if zero or more than 1 entities don't have a relation
    withoutRelationCounter = list(dictRelations.values()).count(None)
    if  withoutRelationCounter != 1 :
        dictWithoutRelations = {index : relation for (index,relation) in dictRelations.items() if relation == None}
        if withoutRelationCounter == 0 :
            dictWithoutRelations = dictEntities
        INDEX_MAIN_ENTITY = findIndexMainEntity(processed_entry, dictWithoutRelations, indexAction)

    # if only one entity is not attached to a relation keyword, it's the main one
    else :
        INDEX_MAIN_ENTITY = [index for index,relation in dictRelations.items() if relation == None][0]
    print("index_main_entity :", INDEX_MAIN_ENTITY)

    # the hierarchy position : 0 is site, 3 is rack... etc
    hierarchyPosition = {index : list(wiki.ENTITIES.keys()).index(entity) for index, entity in dictEntities.items()}
    finalRelations = dict()

    # for all entities except the main one, we assign the related entity according to the relation
    for index, relation in dictRelations.items() : 
        if index == INDEX_MAIN_ENTITY:
            continue
        for token in processed_entry[index].ancestors :
            if token.i in dictEntities.keys(): 
                if testRelation(token.i, index, relation, hierarchyPosition):
                    finalRelations[index] = (token.i, relation)
                else:
                    finalRelations[index] = (token.i, "ERROR")
                break
            elif token.i == indexAction:
                if testRelation(INDEX_MAIN_ENTITY, index, relation, hierarchyPosition):
                    finalRelations[index] = (INDEX_MAIN_ENTITY, relation)
                else:
                    finalRelations[index] = (INDEX_MAIN_ENTITY, "ERROR")
                break
        if not index in finalRelations :
            finalRelations[index] = (INDEX_MAIN_ENTITY, relation)

    return finalRelations

In [ ]:
def name(processed_entry : Doc, dictioEntities : dict, listNameSynonyms : list, indexesMain : dict) -> int :

    def isName(token : Token) -> bool :
        if token.is_upper or token.pos_ == "PROPN" :
            return True
        return False   

    def findClose(processed_entry : Doc, index : int) -> (int|None) :
        if index +1 <= len(processed_entry)-1 and isName(processed_entry[index+1]) :
            return index+1
        if 0 <= index -1 and isName(processed_entry[index-1]):
            return index-1
        return None

    def findAttachedEntity(processed_entry : Doc, index : int) -> (int|None) : 
        counter = 0
        for token in processed_entry[index].ancestors :
            if counter == 3 :
                break
            if token.i in dictioEntities.keys() and token.i not in dictioEntityNames.keys() :
                return token.i
            if token.i == indexesMain["action"] :
                return indexesMain["entity"]
            counter += 1
        return None

    EXPLICIT =  PARAMETERS_DICT["name"]
    IMPLICIT = ["current","main"]

    dictioEntityNames = {} # dict with entityIndex : NameOfTheEntityIndex

    # begin with the synonyms of "called"
    for nameSynonymIndex in listNameSynonyms :
        currentToken = processed_entry[nameSynonymIndex]
        attachedEntityIndex = findAttachedEntity(processed_entry, nameSynonymIndex)
        attachedValueIndex = None

        if attachedEntityIndex == None or attachedEntityIndex in dictioEntityNames.keys() :
            continue

        if (currentToken.similarity(nlp("called")[0]) > SIMILARITY_THRESHOLD
            and currentToken.i < len(processed_entry)-1
            and currentToken.is_ancestor(processed_entry[currentToken.i +1])) :
            attachedValueIndex = currentToken.i +1

        if len(list(currentToken.children)) != 0  and attachedValueIndex == None :
            for token in currentToken.rights :
                if isName(token) :
                    attachedValueIndex = token.i
                    break
            if attachedValueIndex == None :
                for token in currentToken.lefts :
                    if isName(token) :
                        attachedValueIndex = token.i # we choose the rightmost

        if len(list(currentToken.ancestors)) != 0 and attachedValueIndex == None :
            counter = 0
            for token in currentToken.ancestors :
                if counter == 2 :
                    break
                if isName(token) :
                    attachedValueIndex = token.i
            
        if attachedValueIndex != None and attachedValueIndex not in dictioEntityNames.values() :            
            dictioEntityNames[attachedEntityIndex] = attachedValueIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        # if the name if right beside the entity
        for entityIndex,_ in dictioEntities.items() :
            if entityIndex in dictioEntityNames.keys() :
                continue
            attachedValueIndex = findClose(processed_entry, entityIndex)
            if attachedValueIndex != None and attachedValueIndex not in dictioEntityNames.values() :
                dictioEntityNames[entityIndex] = attachedValueIndex

    if len(dictioEntityNames) < len(dictioEntities) : # if not all names found

        for token in processed_entry : # look directly for a name

            if isName(token) :
                indexAttachedEntity = findAttachedEntity(processed_entry, token.i)
                if (indexAttachedEntity not in dictioEntityNames.keys()
                    and token.i not in dictioEntityNames.values()
                    and indexAttachedEntity != None) :
                    dictioEntityNames[indexAttachedEntity] = token.i

    if len(dictioEntityNames) < len(dictioEntities) : # if still not all names found
        pass
    # TODO : get current names

    # TODO : check for implicit words now
    
    return dictioEntityNames

In [ ]:
def associateParameters(processed_entry : Doc, KEY_WORDS_ENTRY : dict, dictEntities : dict, dictioEntityNames : dict) -> dict :
    association = {}

    if len(dictEntities) == 1:
        for index, keyword in KEY_WORDS_ENTRY.items():
            if keyword in PARAMETERS_DICT.keys():
                association[index] = (INDEX_MAIN_ENTITY, keyword)
        return association
    
    for index, keyword in KEY_WORDS_ENTRY.items():
        flagFor = False
        if keyword in PARAMETERS_DICT.keys():

            if len(dictEntities) == len(dictioEntityNames):

                if keyword != "name":
                    for token in processed_entry[index].subtree:
                        if token.lower_ == "for":
                            flagFor = True
                        if token.i in dictEntities.keys() and flagFor == True:
                            association[index] = (token.i, keyword)
                            break

                else:
                    for ancestor in processed_entry[index].ancestors:
                        if ancestor.i in dictEntities.keys() and not((ancestor.i, keyword) in association.values()):
                            association[index] = (ancestor.i, keyword)
                            break

            else:

                if keyword == "name":
                    for ancestor in processed_entry[index].ancestors:
                        if ancestor.i in dictEntities.keys() and not((ancestor.i, keyword) in association.values()):
                            association[index] = (ancestor.i, keyword)
                            break

                else:

                    for ancestor in processed_entry[index].ancestors:
                        if (ancestor.i, "name") in list(association.values()):
                            continue

                        if ancestor.i in KEY_WORDS_ENTRY.keys() and KEY_WORDS_ENTRY[ancestor.i] == "name":
                            association[index] = ([ancestor.i], keyword)
                            break

                        if ancestor.i in dictEntities.keys() and not((ancestor.i, keyword) in association.values()):
                            association[index] = (ancestor.i, keyword)
                            break

            # if not index in association.keys():
            #     for token in processed_entry[index].subtree:
            #         if token.i in dictEntities.keys():
            #             association[index] = (token.i, keyword)
            #             break
            
            if not index in association.keys():
                association[index] = (INDEX_MAIN_ENTITY, keyword)


    for index, (index2, parameterType) in association.items():
        if type(index2) == list:
            association[index] = (association[index2[0]][0], parameterType)
            
    return association

In [ ]:
def buildFullName(processed_entry : Doc, dictioEntityNames : dict, dictEntities, finalRelations : dict, entityIndex : int, EXISTING_ENTITY_NAMES : dict) -> str :
    """Build the full name of the entity specified"""

    # Start with the partial name : the name of the entity specified
    partialName = str(processed_entry[dictioEntityNames[entityIndex]])
    # List that contains all the entity index that are part of the name
    parentalTreeIndexList = [entityIndex]
    # Dictionnary that gives a level of hierarchy to an entity. Ex : index_of_a_site : 0, index_of_a_building : 1
    hierarchyPosition = {index : list(wiki.ENTITIES.keys()).index(entity) for index, entity in dictEntities.items()}
    levelCounter = hierarchyPosition[entityIndex]
    startingLevel = hierarchyPosition[entityIndex] - 1

    # Check if the name has been written with / character 
    if "/" in partialName:
        if partialName[0] == "/":
            if partialName[-1] == "/":
                partialName = partialName[:-1]
        else:
            if partialName[-1] == "/":
                partialName = "/" + partialName[:-1]
            else:
                partialName = "/" + partialName
        for existingName in EXISTING_ENTITY_NAMES.keys():
            if EXISTING_ENTITY_NAMES[existingName] == dictEntities[entityIndex] and partialName == existingName[-len(partialName):]:
                return existingName
        nEntity = partialName.count("/")
        startingLevel -= (nEntity-1)
        print(partialName, startingLevel)
        if startingLevel == -2:
            return partialName

    # Go through all the levels needed for the name
    for level in range(startingLevel, -1, -1):
        temporaryIndex = None
        
        # Go through all the relations that exists between entities
        for indexParent, (indexSon, relationType) in finalRelations.items():

            # The relation has the good parent if (3 conditions) : 
            #      the son of the relation is in our current list,
            #      the level of the parent is the one that we are looking for,
            #      the type of the relation is "hierarchy"
            if indexSon in parentalTreeIndexList and hierarchyPosition[indexParent] == level and relationType == "hierarchy":
                parentalTreeIndexList.append(indexParent)
                partialName = str(processed_entry[dictioEntityNames[indexParent]]) + "/" + partialName
                levelCounter -= 1
                break
            # If the son of the current relation is not in the list but satisfy every other condition and the son is the MAIN_ENTITY, the parent is kept in extreme emergency.
            if indexSon == INDEX_MAIN_ENTITY and hierarchyPosition[indexParent] == level and relationType == "hierarchy":
                temporaryIndex = indexParent

        # If not parent found
        if levelCounter != level:
            # Search for existing entity with the same partial name
            for existingName in EXISTING_ENTITY_NAMES.keys():
                if EXISTING_ENTITY_NAMES[existingName] == dictEntities[entityIndex] and partialName == existingName[-len(partialName):]:
                    return existingName
            # Extreme emergency : assuming that specified entity and MAIN_ENTITY have the same parental tree
            if temporaryIndex != None:
                parentalTreeIndexList.append(temporaryIndex)
            # Informations are incomplete
            else:
                return None
    
    partialName = "/P/" + partialName
    # TODO : verifiy this upper function (needed or not)
    return partialName.upper()

# TODO : name with / 
# TODO : device level : might be multiple OR adapt hierarchyPosition
# TODO : if there is hole in the tree, verify if the next parent as only one son

In [ ]:
# TODO : the similarity func is very time-taking, we must shorten the process time or find another way

def main() -> str :
    FINAL_INSTRUCTION = ""

    # TODO : add already existing entity names
    ENTITIES_FULL_NAME = {"entity" : list(wiki.ENTITIES.keys())}
    KEY_WORDS_ALL = {**ENTITIES_FULL_NAME,  **PARAMETERS_DICT}

    EXISTING_ENTITY_NAMES = scrapping.scrapAllName("DEMO_BASIC.ocli")

    natural_entry = input("Enter a prompt. Please follow the instructions given.\n")
    processed_entry = nlp(natural_entry)

    KEY_WORDS_ENTRY = {} 
    # we detect key words in the sentence given and put them into KEY_WORDS_ENTRY
    lastParameter = None
    for index,token in enumerate(processed_entry) :
        matching_list = [] # list of tuples with the similarity score and type of key word (for each key word)
        # TODO : name without entity specified
        if token.pos_ == "VERB" and str(token) == token.lemma_ and token.head == token : # 2nd test : if infinitive verb
            for parameter in ACTIONS_DEFAULT.keys() :
                if token.lower_ in ACTIONS_DEFAULT[parameter] :
                    matching_list.append((1,parameter))
                else :
                    similarity = max([token.similarity(nlp(word)[0]) for word in ACTIONS_DEFAULT[parameter]])
                    matching_list.append((similarity,parameter))
        else :
            for parameter in KEY_WORDS_ALL.keys() :
                if token.lower_ in KEY_WORDS_ALL[parameter] :
                    matching_list.append((1,parameter))
                elif token.pos_ in ["NOUN","ADP","VERB"] :
                    similarity = max([token.similarity(nlp(word)[0]) for word in KEY_WORDS_ALL[parameter]])
                    matching_list.append((similarity,parameter))
        
        if not matching_list :
            continue

        match = max(matching_list)

        # if "called" or a synonym is used for a parameter and not for an entity
        if match[1] == "name" and (lastParameter == token.head or lastParameter in token.children) :
            continue
        if match[1] == lastParameter and match[1] != "name" :
            continue
        if match[0] > SIMILARITY_THRESHOLD :
            # if is considered a key word, is added to the dict
            KEY_WORDS_ENTRY[index] = match[1] 
            if match[1] in PARAMETERS_DICT.keys() :
                lastParameter = match[1]

    print(KEY_WORDS_ENTRY)

    dictEntities = {index : processed_entry[index].text for index,keyword in KEY_WORDS_ENTRY.items() if keyword == "entity"}
    print(dictEntities)

    # test detection
    list_key_param = list(KEY_WORDS_ENTRY.values())
    count_action = 0 # the nb of action words indentified
    for action_type in ACTIONS_DEFAULT.keys() :
        count_action += list_key_param.count(action_type)

    if count_action != 1 :
        raise Exception("Action not detected")
    
    # TODO : check entities
    # if no entity mentionned : must be alteration
    # if no entity and color, seek for keyword reserved etc besides the color

    # if no entity :check the ocli file
    global INDEX_ACTION
    global INDEX_MAIN_SUBJECT
    INDEX_ACTION = [index for index,keyword in KEY_WORDS_ENTRY.items() if keyword in ACTIONS_DEFAULT.keys()][0]
    finalRelations = findRelations(processed_entry, dictEntities, INDEX_ACTION)
    print("finalRelations : ", finalRelations)
    INDEX_MAIN_SUBJECT = findIndexMainSubject(processed_entry, KEY_WORDS_ENTRY, INDEX_ACTION, INDEX_MAIN_ENTITY)  
    print("index main subject : ", INDEX_MAIN_SUBJECT)

    INDEXES_MAIN = {"subject" : INDEX_MAIN_SUBJECT, 
                    "action" : INDEX_ACTION, 
                    "entity" : INDEX_MAIN_ENTITY}

    dictioEntityNames = name(processed_entry,
                             dictEntities,
                             [index for index,parameter in KEY_WORDS_ENTRY.items() if parameter == "name"],
                             INDEXES_MAIN)
    print("names : ",dictioEntityNames)

    if INDEX_MAIN_SUBJECT not in KEY_WORDS_ENTRY.keys() :
        # TODO : the dectection is different (key word set TO)
        pass
    
    association = associateParameters(processed_entry, KEY_WORDS_ENTRY, dictEntities, dictioEntityNames)
    print("association between keyword and entity :", association)

    fullName = buildFullName(processed_entry, dictioEntityNames, dictEntities, finalRelations, INDEX_MAIN_ENTITY, EXISTING_ENTITY_NAMES)
    print(fullName)
    if fullName == None:
        raise ValueError("Not all the parent tree is known to name the object.")
    
    # TODO : change the name of parameters depending on the entity (e.g. unit -> floorUnit)
    if KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT] == "entity" :
        # we do the processes related to each parameter

        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" :
            dictioEntityParameters = wiki.makeDictParam(processed_entry[INDEX_MAIN_SUBJECT].text)
            dictioEntityParameters["name"] = fullName
            allEntryItemsList = list(KEY_WORDS_ENTRY.items())
            for counter,(index,parameter) in enumerate(allEntryItemsList) :
                if ((not parameter in PARAMETERS_DICT.keys()) 
                    or bool(dictioEntityParameters[parameter]) == True 
                    or association[index][0] != INDEX_MAIN_SUBJECT) :
                    continue
                lastKeyWordIndex = 0 if counter == 0 else allEntryItemsList[counter-1][0]
                nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItemsList)-1 else allEntryItemsList[counter+1][0]
                # get the parameter value
                # TODO : change to the get file
                parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
                dictioEntityParameters[parameter] = parameterValue # store the value
            print(dictioEntityParameters)
            FINAL_INSTRUCTION = tools.create(dictEntities[INDEX_MAIN_SUBJECT], dictioEntityParameters)

        elif KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_NEGATIVE" :
            # REQUIRES : just the full name
            FINAL_INSTRUCTION = tools.delete("", {"name" : fullName})
        
        else:
            raise NotImplementedError("The action '"+KEY_WORDS_ENTRY[INDEX_ACTION]+"' has not been implemented for '"+KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT]+"' as main subject")

    else:
        # TODO : if parameter is "name"
        if KEY_WORDS_ENTRY[INDEX_ACTION] == "ACTION_POSITIVE" :
            
            allEntryItemsList = list(KEY_WORDS_ENTRY.items())
            for counter,(index,parameter) in enumerate(allEntryItemsList) :
                if (not parameter in PARAMETERS_DICT.keys()) or parameter == "name": # or association[index][0] != INDEX_MAIN_ENTITY
                    continue
                lastKeyWordIndex = 0 if counter == 0 else allEntryItemsList[counter-1][0]
                nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItemsList)-1 else allEntryItemsList[counter+1][0]
                # get the parameter value
                parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
                fullName = buildFullName(processed_entry, dictioEntityNames, dictEntities, finalRelations, association[index][0])
                if fullName == None:
                    raise ValueError("Not all the parent tree is known to name the object.")
                FINAL_INSTRUCTION += tools.createAttribute(fullName, parameter, parameterValue) + "\n"

        elif KEY_WORDS_ENTRY[INDEX_ACTION] == "ALTERATION" :
            # REQUIRES : the full name, the parameter and its value
            
            allEntryItemsList = list(KEY_WORDS_ENTRY.items())
            for counter,(index,parameter) in enumerate(allEntryItemsList) :
                if (not parameter in PARAMETERS_DICT.keys()) or parameter == "name": # or association[index][0] != INDEX_MAIN_ENTITY
                    continue
                lastKeyWordIndex = 0 if counter == 0 else allEntryItemsList[counter-1][0]
                nextKeyWordIndex = len(processed_entry) if counter == len(allEntryItemsList)-1 else allEntryItemsList[counter+1][0]
                # get the parameter value
                parameterValue = globals()[parameter](processed_entry, index, lastKeyWordIndex, nextKeyWordIndex)
                fullName = buildFullName(processed_entry, dictioEntityNames, dictEntities, finalRelations, association[index][0])
                if fullName == None:
                    raise ValueError("Not all the parent tree is known to name the object.")
                FINAL_INSTRUCTION += tools.setAttribute(fullName, parameter, parameterValue) + "\n"
        
        else:
            raise NotImplementedError("The action '"+KEY_WORDS_ENTRY[INDEX_ACTION]+"' has not been implemented for '"+KEY_WORDS_ENTRY[INDEX_MAIN_SUBJECT]+"' as main subject")
    
    # if seeking the name for the main entity, pass the indexaction as parameter
    # if no name found, check the type of action : if +, a name is needed, otherwise not necessarily

    # check if parameters were not given
        
    print("Final instruction :", FINAL_INSTRUCTION)

    # return KEY_WORDS_ENTRY

In [ ]:
testNlp5 = nlp("put a rack in a room named A2 which is called R1 with dimension 3 4 2 rotated by 45 45 45")
"set the name of the rack R1 to R2"
"create a rack A1 with dimension 5 5 5 in meters in the room R2"
"create the size 5 5 5 for the building B1 in the site S1 and the size 4 6 7 for the building B2"
"create the room named /P/BASIC/B1/A1"
main()

In [ ]:
nlp("name")[0].similarity(nlp("called")[0])
# TODO : nlp("dimensions")[0].similarity(nlp("complexity")[0])
# [nlp("current")[0].similarity(nlp(word)[0]) for word in ["axis", "orientation"]]

In [ ]:
#Test position, size, rotation, template
testNlp1 = nlp("create the building named BATIMENT with the dimension 50.4 40 60, turned by 90 degrees and with the template called intel640")
testNlp2 = nlp("make a room called Nowhere in the building IMT next to the building named Centrale")
testNlp3 = nlp("set the complexity of the room R1 to easy")
testNlp4 = nlp("in the site IMT, create a room with R1 as name")
testNlp5 = nlp("put a rack in a room named A2 which is called R1 with dimension 3 4 2 rotated by 45 45 45")
testNlp6 = nlp("create the parameter codepostal for the room R1 with value 01170")
testNlp7 = nlp("put a rack called R1 in a room named A2 with dimension 3 4 2 rotated by 45 45 45")
testNlp8 = nlp("in the site S1, in the building B1, create a room R1")
testNlp9 = nlp("put a rack called R1 in a room with dimension 3 4 2 rotated by 45 45 45")
testNlp10 = nlp("put a rack in a room named A2 which is called R1 at 0 0 with dimension 3 4 2 rotated by 45 45 45 in the building B1 in the site S1")
testNlp11 = nlp("create a rack A1 with dimension 5m 5m 5m in the room R2")
testNlp12 = nlp("create the size 5 5 5 for the building B1 in the site S1 and the size 4 6 7 for the building B2")
testNlp13 = nlp("create a room called R1 with imb-ns1200 template")
# print("Position :" + str(position(testNlp, 0, 0, 4, "building")))
# print("Rotation :" + str(rotation(testNlp, 15, 11, 19, "building")))
# print("Dimension :" + str(size(testNlp, 10, 6, 14, "building")))
# print("Template :" + str(template(testNlp, 22, 19, 23, "building")))


In [ ]:
spacy.displacy.render(testNlp12, style="dep")